# EP4 - Simulação paralelizada de sólidos usando DEM

Nota: todos os nomes de funções e etc se encontram em inglês. Isso foi feito por conta do próprio python, que é inteiramente em inglês. 

## Preâmbulo

In [13]:
import math
import numpy as np
import pandas as pd
from numpy.linalg import norm
from itertools import combinations
import timeit


In [14]:
# Algumas funções geométricas auxiliares

def vec(x,y,z)-> np.ndarray:
    '''Função para facilmente transformar coordenadas em um vetor numpy.'''
    return np.array([x,y,z],dtype=float)

def rotation_3d(vec: np.ndarray, rot: iter)-> np.ndarray:
    '''Função que rotaciona um vetor 3d em torno dos eixos x,y,z, a partir
    de um vetor de rotação rot = [rot_x, rot_y, rot_z]. Nota: @ é um operador
    da biblioteca numpy que realiza a multiplicação de matrizes.'''

    if rot[0] != 0:
        Rx = np.array([[1,0,0],
                       [0,np.cos(rot[0]),-np.sin(rot[0])],
                       [0,np.sin(rot[0]),np.cos(rot[0])]])
        vec = Rx @ vec
    
    if rot[1] != 0:
        Ry = np.array([[np.cos(rot[1]),0,np.sin(rot[1])],
                       [0,1,0],
                       [-np.sin(rot[1]),0,np.cos(rot[1])]])
        vec = Ry @ vec

    if rot[2] != 0:
        Rz = np.array([[np.cos(rot[2]),-np.sin(rot[2]),0],
                       [np.sin(rot[2]),np.cos(rot[2]),0],
                       [0,0,1]])
        vec = Rz @ vec 
        
    return vec

In [15]:
# Classe das Partículas (Grãos)

class grain: 
    '''Classe que define uma partícula, que aqui chamo de grain (grão) por simplicidade.
define um grão por sua posição, raio, massa, velocidade, aceleração e força. Para 
gerenciar os grãos, é sempre criada uma lista de grãos, grain_list, que é argumento
de todas as funções que gerenciam as interações que envolvem os grãos.'''

    def __init__(self, pos, radius, density, vel): 
        self.radius = float(radius)
        x,y,z = pos
        self.pos     = vec(x,y,z)
        self.mass    = density*math.pi*(self.radius**3)*(4/3)
        self.vel     = vel
        self.acc     = vec(0.,0.,0.)
        self.force   = vec(0.,0.,0.)

In [16]:
# Funções de gerenciamento de forças e interações


def cube_walls(center: iter,
               scale: float,
               grain_list: iter,
               K: float,
               rot = np.array([0.,0.,0.])) -> None:
    '''Função que gerencia a geometria do cubo, e a força de contato nas 
    partículas que estão em contato com as paredes do cubo.'''

    x,y,z = center

    vectors = [vec(1,0,0), vec(0,1,0), vec(0,0,1)]

    if not rot.all == 0:
        rot_vectors = [rotation_3d(x, rot) for x in vectors]
        vectors = rot_vectors.copy()
    
    n_vectors = [x/norm(x) for x in vectors]
    vectors = n_vectors.copy()

    t_matrix = np.array(vectors).T

    inv_matrix = np.linalg.inv(t_matrix)

    for grain in grain_list:

        p = grain.pos

        p = t_matrix @ p

        if p[0] > x + scale - grain.radius:
            p[0] = x +  scale - grain.radius
            grain.vel[0] = -grain.vel[0]*K

        if p[0] < x - scale + grain.radius:
            p[0] = x - scale + grain.radius
            grain.vel[0] = -grain.vel[0]*K
        
        if p[1] > y + scale - grain.radius:
            p[1] = y + scale - grain.radius
            grain.vel[1] = -grain.vel[1]*K
        
        if p[1] < y - scale + grain.radius:
            p[1] = y - scale + grain.radius
            grain.vel[1] = -grain.vel[1]*K

        if p[2] > z + scale - grain.radius:
            p[2] = z + scale - grain.radius
            grain.vel[2] = -grain.vel[2]*K
        
        if p[2] < z - scale + grain.radius:
            p[2] = z - scale + grain.radius
            grain.vel[2] = -grain.vel[2]*K

        p = inv_matrix @ p

        grain.pos = p

def contact(gr1: grain, gr2: grain) -> None:
    '''Função que gerencia as forças de contato entre dois grãos.'''

    # Aqui, reduzi os valores ks e kd para que os grãos estabilizem mais rápido.

    ks = 1e2
    kd = 1e2
    kt = 0.5
    mu = 0.2

    # Força Normal
    
    csi = max(0, gr1.radius + gr2.radius - norm(gr1.pos - gr2.pos))

    N = gr1.pos - gr2.pos

    Fs = ks*csi*N/norm(N)

    Fd = kd*N

    Fn = Fs + Fd

    gr1.force += Fn
    gr2.force -= Fn

    # Força Tangencial

    Vt = gr1.vel - gr2.vel


    Ft = -min(mu*norm(Fn), kt*norm(Vt))*Vt/norm(Vt)

    gr1.force += Ft
    gr2.force -= Ft


def apply_gravity(grain_list: iter, g: float) -> None:
    '''Função que gerencia a força gravitacional aplicada somente aos grãos.'''
    for gr in grain_list:
        gr.force += gr.mass*g

def reset_force(grain_list: iter) -> None:
    '''Função que coloca as forças de todos os grãos em zero. A finalidade
    dessa função é simples: se as forças não forem todas colocadas em zero
    no início de cada iteração, forças como a gravidade iriam se acumular
    de uma iteração para a outra, algo não desejado.'''
    for gr in grain_list:
        gr.force = vec(0., 0., 0.)

def verlet(grain_list: iter, dt: float) -> None:
    '''Função que aplica a integração de verlet para atualizar as posições
    dos grãos.'''
    for gr in grain_list:
        a = gr.force/gr.mass
        gr.vel += (gr.acc + a) * (dt/2.)
        gr.pos += gr.vel * dt + 0.5*a*(dt**2.)
        gr.acc  = a

def apply_contacts(grain_list: iter) -> None:
    '''Função que verifica se houve contato entre cada um dos pares de grãos
    por meio do cálculo de suas distâncias e, caso haja, aplica a função de
    força de contato contact().'''

    for gr1, gr2 in combinations(grain_list, 2):
        if norm(gr1.pos - gr2.pos) < gr1.radius + gr2.radius:
            contact(gr1, gr2)

def time_loop(grain_list: iter,
              dt: float,
              g: float,
              center: iter,
              scale: float,
              rot: iter,
              K: float) -> None:
    '''Função que gerencia o laço temporal, aplicando cada uma das funções
    previamente definidas em ordem para cada iteração. Retorna os tempos de
    execução das funções mais custosas, para fins de análise de desempenho.'''
    
    reset_force(grain_list)
    apply_gravity(grain_list, g)

    time = timeit.default_timer()
    apply_contacts(grain_list)
    time_contacts = timeit.default_timer() - time

    time = timeit.default_timer()
    cube_walls(center, scale, grain_list, K, rot)
    time_walls = timeit.default_timer() - time

    verlet(grain_list, dt)

    return time_contacts, time_walls

In [17]:
# Funções auxiliares de geração de grãos

def generate_grains_random(n: iter,
                           radius: float,
                           density: float,
                           sidelength: float) -> list:
    '''Função que gera uma lista de grãos com posições e velocidades aleatórias,
    respeitando a escala do cubo e o raio dos grãos.'''
    
    grain_list = []
    sidelength -= 1*radius
    for _ in range(n):
        gr = grain(vec(np.random.uniform(-sidelength/2, sidelength/2),
                       np.random.uniform(-sidelength/2, sidelength/2), 
                       np.random.uniform(0, sidelength)),
                    radius,
                    density,
                    vec(np.random.uniform(-1, 1),
                        np.random.uniform(-1, 1),
                        np.random.uniform(-1, 1)))
        grain_list.append(gr)
    return grain_list

def generate_grains_uniform(cube_scale: iter,
                            n_max: iter,
                            radius: float,
                            density: float,
                            randvel = False) -> list:
    '''Função que gera os grãos uniformemente distribuídos em uma grade, respeitando a escala
    do cubo e o raio dos grãos. É possível escolher uma escala menor que a do cubo e um número
    arbitrário de grãos, desde que caibam dentro do cubo com espaço. É possível também escolher
    se as velocidades iniciais serão todas nulas ou aleatórias.'''

    grain_list = []

    reduced_scale = cube_scale - radius

    n_max = [min(int(r/radius), n) for r, n in zip(reduced_scale, n_max)]

    xs, ys, zs = reduced_scale

    for x in np.linspace(-xs, xs, n_max[0]):
        for y in np.linspace(-ys, ys, n_max[1]):
            for z in np.linspace(-zs, zs, n_max[2]):
                gr = grain(vec(x,y,z), radius, density, vec(0.,0.,0.))
                grain_list.append(gr)
    
    if randvel:
        for gr in grain_list:
            gr.vel = vec(np.random.uniform(-1, 1),
                        np.random.uniform(-1, 1),
                        np.random.uniform(-1, 1))

    return grain_list

In [18]:
# Funções auxiliares de movimentação do cubo

def linear_shake(center: iter,
                 axis: int,
                 T: float,
                 dt: float,
                 amplitude = 1.,
                 omega = 1.) -> list:
    '''Função que translada o cubo em um eixo, de forma a simular uma vibração de um lado para o outro.'''

    cube_pos = []

    center = np.array(center,dtype=float)

    total_angle = 2*np.pi * T

    n_elements = np.arange(0,T,dt).shape[0]

    angle_range = np.linspace(0, total_angle,n_elements)

    omega = omega/(2*np.pi)

    for theta in angle_range:
        center[axis] = amplitude*np.sin(omega*theta)
        cube_pos.append(center.copy())

    return np.array(cube_pos)

def angular_shake(rot: iter,
                  axis: int,
                  T: float,
                  dt: float,
                  amplitude = 1.,
                  omega = 1.) -> list:
    '''Função que rota o cubo em um eixo e depois volta, de forma a simular uma vibração com rotação.'''

    cube_rot = []

    rot = np.array(rot,dtype=float)

    amplitude = amplitude * (np.pi/2) # Amplitude em radianos. Logo, amplitude = 1 -> rotação de 90°
    
    n_elements = np.arange(0,T,dt).shape[0]

    total_angle = 2*np.pi * T

    angle_range = np.linspace(0, total_angle, n_elements)

    omega = omega/(2*np.pi)

    for theta in angle_range:
        rot[axis] = amplitude*np.sin(omega*theta)
        cube_rot.append(rot.copy())

    return np.array(cube_rot)

def linear_shake_wait(center: iter,
                  axis: int,
                  T: float,
                  dt: float,
                  amplitude = 1.,
                  omega = 1.,
                  wait_time_before = .5,
                  wait_time_after = .5) -> list:
    '''Função que mantém o cubo fixo por alguns segundos, depois o move, e finalmente o mantém fixo novamente.
    Esses tempos de espera são para estabilização dos grãos.'''

    cube_pos = []

    wait_before = np.zeros(shape = (np.arange(0,wait_time_before,dt).shape[0],3))

    for w in wait_before:
        cube_pos.append(w)

    rot_time = T - wait_time_after - wait_time_before

    pos = linear_shake(center, axis, rot_time, dt, amplitude, omega)

    for p in pos:
        cube_pos.append(p)

    wait_after = np.zeros(shape = (np.arange(0,wait_time_after,dt).shape[0],3))
    
    for w in wait_after:
        cube_pos.append(w)

    return np.array(cube_pos)

def angular_shake_wait(rot: iter,
                  axis: int,
                  T: float,
                  dt: float,
                  amplitude = 1.,
                  omega = 1.,
                  wait_time_before = .5,
                  wait_time_after = .5) -> list:
    '''Função que mantém o cubo fixo por alguns segundos, depois o rotaciona, e finalmente o mantém fixo novamente.
    Esses tempos de espera são para estabilização dos grãos.'''

    cube_rot = []

    wait_before = np.zeros(shape = (np.arange(0,wait_time_before,dt).shape[0],3))

    for w in wait_before:
        cube_rot.append(w)

    rot_time = T - wait_time_after - wait_time_before

    rot = angular_shake(rot, axis, rot_time, dt, amplitude, omega)

    for r in rot:
        cube_rot.append(r)

    wait_after = np.zeros(shape = (np.arange(0,wait_time_after,dt).shape[0],3))
    
    for w in wait_after:
        cube_rot.append(w)

    return np.array(cube_rot)

In [19]:
def sim(grain_list, time_steps, dt, g, center, scale, rot, K, cube_rot, cube_pos):
    pos_list = [] # Lista de posições dos grãos

    time_contacts_list = [] # Lista de tempos de execução da função apply_contacts()
    time_walls_list = [] # Lista de tempos de execução da função cube_walls()
    time_loop_list = [] # Lista de tempos de execução do laço temporal

    time = timeit.default_timer()

    for i, _ in enumerate(time_steps):

        looptime = timeit.default_timer()

        time_contacts, time_walls = time_loop(grain_list, dt, g, center, scale, rot, K)
        
        temp_list = []
        
        for gr in grain_list:
            for pos in np.array(gr.pos):
                temp_list.append(pos)

        center = cube_pos[i]
        rot = cube_rot[i]
        
        pos_list.append(temp_list.copy())

        time_contacts_list.append(time_contacts)
        time_walls_list.append(time_walls)
        time_loop_list.append(timeit.default_timer() - looptime)

    total_time = timeit.default_timer() - time

    return np.array(pos_list), np.array(time_contacts_list), np.array(time_walls_list), np.array(time_loop_list), total_time

## Execução de uma simulação de exemplo

In [20]:
# Parâmetros

T = 2.5

dt = 0.001

K = .001

g = np.array([0.,0.,-9.81])

# Geometria do cubo

center = np.array((0.,0.,0.))

scale = 1

rot = np.array((0.,0.,0.))

time_steps = np.arange(0,T,dt)

# Grãos

cube_scale = np.array((1.,1.,1.))

n_max = (5,5,5)

radius = 0.05

density = 1600

randvel = False

grain_list = generate_grains_uniform(cube_scale, n_max, radius, density, randvel=randvel)

cube_pos = linear_shake_wait(center = center,
                 axis = 2,
                 T = T,
                 dt =  dt,
                 omega=0,
                 amplitude=.5)

cube_rot = angular_shake_wait(rot = rot, 
                         axis = 0,
                         T = T,
                         dt = dt,
                         omega = 10,
                         amplitude=.5)

cube_rot += angular_shake_wait(rot = rot, 
                         axis = 1,
                         T = T,
                         dt = dt,
                         omega = 10,
                         amplitude=.5)

In [21]:
# Execução da simulação

pos_list, time_contacts_list, time_walls_list, time_loop_list, total_time = sim(grain_list, time_steps, dt, g, center, scale, rot, K, cube_rot, cube_pos)

In [22]:
total_time

65.39843539998401

In [23]:
np.average(time_contacts_list)

0.02420156856011599

In [24]:
np.average(time_walls_list)

0.0006935968409758061

In [25]:
np.average(time_loop_list)

0.02615829711984843

## Extração dos Dados

In [26]:
# Criando arquvios .csv completos para as posições dos grãos e do cubo

df = pd.DataFrame(pos_list)

df.to_csv('pos.csv')

pos_df = pd.DataFrame(cube_pos)

rot_df = pd.DataFrame(cube_rot)

cube_df = pd.merge(pos_df, rot_df, left_index=True, right_index=True)

cube_array = np.array(cube_df)

cube_df.to_csv('cube.csv')

In [27]:
'''Precisamos cortar alguns frames para poder exportar ao Blender. O motivo para isso é que as animações
precisam ser renderizadas em no máximo 60 fps, mais do que isso é desperdício de poder computacional.
Então, cortamos a lista de quadros de forma a pegar um quadro a cada 60 quadros, ou seja, 60 quadros
por segundo. Essas listas serão carregadas no blender.'''

persec = 1/dt

fps = 60

frame_step = int(round(persec/fps,0))

cut_frames = pos_list[::frame_step]

df_frames = pd.DataFrame(cut_frames)

df_frames.to_csv('cut_frames.csv')

cube_cut_frames = cube_array[::frame_step]

cube_df_frames = pd.DataFrame(cube_cut_frames)

cube_df_frames.to_csv('cube_cut_frames.csv')

Caso seja de interesse, posso disponibilizar o código interno no Blender que gera as animações.